In [1]:
from fastai.vision import *
import onnx
import onnxruntime
from PIL import Image

In [2]:
learn = load_learner('.')
fastai_model = learn.model.eval()

In [3]:
imagenet_mean, imagenet_std = torch.tensor([[0.485, 0.456, 0.406], [0.229, 0.224, 0.225]])

In [4]:
class ImageScale(nn.Module):
    def forward(self, x):
        xx = (x/255.0-imagenet_mean[...,None,None]) / imagenet_std[...,None,None] 
        return xx.unsqueeze(0)
    
scale_layer = ImageScale()
softmax_layer = torch.nn.Softmax(dim=1)

In [5]:
final_model = nn.Sequential(scale_layer, fastai_model, softmax_layer)

In [6]:
dummy_image = torch.autograd.Variable(torch.randn(3, 256, 256))
model_name = 'pneumonia.onnx'
torch.onnx.export(final_model, dummy_image,
                  model_name, input_names = ['image'], output_names = ['diagnosis'])
onnx_model = onnx.load(model_name)
onnx.checker.check_model(onnx_model)

In [7]:
image = Image.open('pneumonia.jpeg')
image.size, image.mode

((1106, 762), 'L')

In [8]:
image = image.resize((256, 256))
image = image.convert(mode='RGB')
image.size, image.mode

((256, 256), 'RGB')

In [9]:
image_data = np.array(image).transpose(2, 0, 1).astype('float32')
image_data.shape, image_data.dtype

((3, 256, 256), dtype('float32'))

In [10]:
session = onnxruntime.InferenceSession(model_name)

In [11]:
input_name = session.get_inputs()[0].name
input_name

'image'

In [12]:
onnx_result = session.run([], {input_name: image_data})
onnx_result

[array([[0.056997, 0.943003]], dtype=float32)]

In [13]:
image_tensor = torch.tensor(image_data)
fastai_model = learn.model.eval()

In [16]:
input_tensor = scale_layer(image_tensor)
with torch.no_grad():
    fastai_result = fastai_model(input_tensor)
    fastai_result = softmax_layer(fastai_result)

fastai_result

tensor([[0.0570, 0.9430]])